In [22]:
import sqlite3
import pandas as pd
import numpy as np

In [23]:
conn = sqlite3.connect('Db-IMDB.db')
movie = pd.read_sql_query(' SELECT * FROM MOVIE ', conn)


In [3]:
movie.head()

,index,MID,title,year,rating,num_votes
0,0,tt2388771,Mowgli,2018,6.6,21967
1,1,tt5164214,Ocean's Eight,2018,6.2,110861
2,2,tt1365519,Tomb Raider,2018,6.4,142585
3,3,tt0848228,The Avengers,2012,8.1,1137529
4,4,tt8239946,Tumbbad,2018,8.5,7483


# Preprosessing data

## 1.removing duplicates from person table#

In [4]:
actors=pd.read_sql_query("SELECT * FROM PERSON",conn)
actors.shape

(38285, 4)

In [5]:
final=actors.drop_duplicates(subset={"PID"}, keep='first')
final.shape[0]

37566

In [7]:
cursor = conn.cursor()
cursor.execute('CREATE TABLE PERSONS(PID varchar(50) ,Name varchar(50),Gender varchar(10));')
conn.commit()


In [8]:
x=list(final.iloc[0].values)
print(type(x))
print(x)

<class 'list'>
[0, 'nm0000288', ' Christian Bale', 'Male']


In [9]:
from tqdm import tqdm
cursor = conn.cursor()
for i in  tqdm(range(final.shape[0])):
    x=list(final.iloc[i].values)
    cursor.execute('INSERT INTO Persons VALUES (?,?,?);',x[1:])
conn.commit()


100%|██████████████████████████████████████████████████████████████████████████| 37566/37566 [00:04<00:00, 7768.05it/s]


In [10]:
a=pd.read_sql_query('select PID ,COUNT(Name )from PERSONS GROUP BY PID ORDER BY COUNT(Name)  ',conn)
a.head()

,PID,COUNT(Name )
0,None,0
1,nm0000002,1
2,nm0000027,1
3,nm0000039,1
4,nm0000042,1


## 2.Trim PID in M_cast

In [11]:
a=pd.read_sql_query('select  *from M_cast  ',conn)
a.head()

,index,MID,PID,ID
0,0,tt2388771,nm0000288,0
1,1,tt2388771,nm0000949,1
2,2,tt2388771,nm1212722,2
3,3,tt2388771,nm0365140,3
4,4,tt2388771,nm0785227,4


In [12]:
cursor = conn.cursor()
cursor.execute('UPDATE M_Cast SET PID = REPLACE(PID, " ", "")')
conn.commit()

## 3.remove romans characters in  year from movie

In [14]:
cursor = conn.cursor()
cursor.execute('UPDATE Movie SET year = REPLACE(year, "I", "");')
cursor.execute('UPDATE Movie SET year = REPLACE(year, "V", "");')
cursor.execute('UPDATE Movie SET year = REPLACE(year, "X", "");')
conn.commit()


In [15]:
a=pd.read_sql_query('select  *from Movie where year like "I%"  ',conn)
a.head()

,index,MID,title,year,rating,num_votes


# answer 1

In [16]:
a=pd.read_sql_query('select DISTINCT Name,title,year from Movie m INNER JOIN M_Director d ON m.MID=d.MID JOIN PERSONS p ON TRIM(d.PID)=TRIM(p.PID) where m.MID in(select MID from M_Genre where GID in (select GID from GENRE where Name like"%Comedy%")) and ((year%4=0 and year%100!=0) or (year%4=0 and year%100=0 and year%400=0)) ', conn)
a.shape

(232, 3)

In [17]:
a.head()

,Name,title,year
0,Milap Zaveri,Mastizaade,2016
1,Danny Leiner,Harold & Kumar Go to White Castle,2004
2,Anurag Kashyap,Gangs of Wasseypur,2012
3,Frank Coraci,Around the World in 80 Days,2004
4,Griffin Dunne,The Accidental Husband,2008


# answer 2

In [18]:
a=pd.read_sql_query("SELECT Name FROM PERSONS WHERE PID IN(SELECT PID FROM M_Cast WHERE MID IN (SELECT MID  FROM MOVIE WHERE title='Anand'))",conn)
a.head()

,Name
0,Amitabh Bachchan
1,Rajesh Khanna
2,Sumita Sanyal
3,Ramesh Deo
4,Seema Deo


# answer 3

In [19]:
x=pd.read_sql_query("SELECT Name FROM Person WHERE TRIM(PID) IN (SELECT TRIM(PID) FROM MOVIE m INNER JOIN M_cast c ON m.MID=c.MID WHERE  PID IN (SELECT PID FROM MOVIE m INNER JOIN M_cast c ON m.MID=c.MID  WHERE TRIM(year)<'1970') and TRIM(year)>'1990')  ",conn)
x.head()


,Name
0,Rishi Kapoor
1,Amitabh Bachchan
2,Asrani
3,Zohra Sehgal
4,Parikshat Sahni


# answer 4

In [20]:
f=pd.read_sql_query("SELECT Name,COUNT(MID) c FROM M_Director d  JOIN Person p ON d.PID=p.PID GROUP BY Name HAVING c>10 ORDER BY c DESC",conn)
f.head()

,Name,c
0,David Dhawan,39
1,David Dhawan,39
2,Mahesh Bhatt,36
3,Mahesh Bhatt,36
4,Ram Gopal Varma,30


# answer 5

In [21]:
b=pd.read_sql_query("SELECT  year,COUNT(MID) as female_movies from MOVIE WHERE MID IN (SELECT MID FROM M_Cast c JOIN Person p ON c.PID=p.PID WHERE MID IN (SELECT MID FROM M_Cast c JOIN Person p ON c.PID=p.PID WHERE Gender ='Female') and MID NOT IN(SELECT  MID FROM M_Cast c JOIN Person p ON c.PID=p.PID WHERE Gender='Male'))GROUP BY year ",conn)
b

,year,female_movies
0,2018,1
1,1939,1
2,1999,1
3,2000,1


In [22]:
c=pd.read_sql_query("SELECT year,(SELECT  COUNT(MID)  from MOVIE WHERE MID IN (SELECT MID FROM M_Cast c JOIN Person p ON c.PID=p.PID WHERE MID IN (SELECT MID FROM M_Cast c JOIN Person p ON c.PID=p.PID WHERE Gender ='Female') and MID NOT IN(SELECT  MID FROM M_Cast c JOIN Person p ON c.PID=p.PID WHERE Gender='Male'))GROUP BY year)*100/COUNT(MID) AS PERCENTAGE,COUNT(MID) as TOTAL_MOVIES from MOVIE  WHERE year IN (SELECT  year from MOVIE  WHERE MID IN (SELECT MID FROM M_Cast c JOIN Person p ON c.PID=p.PID WHERE MID IN (SELECT MID FROM M_Cast c JOIN Person p ON c.PID=p.PID WHERE Gender ='Female') and MID NOT IN(SELECT  MID FROM M_Cast c JOIN Person p ON c.PID=p.PID WHERE Gender='Male'))) group by year ",conn)
c.head()

,year,PERCENTAGE,TOTAL_MOVIES
0,2018,9,11
1,1939,50,2
2,1999,1,66
3,2000,1,64


# answer 6

In [23]:
a=pd.read_sql_query("select title,count(distinct (PID)) as casts from M_Cast c join MOVIE m ON c.MID=m.MID group by title  order by casts desc",conn )
a.head()

,title,casts
0,Ocean's Eight,238
1,Apaharan,233
2,Gold,215
3,My Name Is Khan,213
4,Captain America: Civil War,191


# answer 7

In [24]:
a=pd.read_sql_query("select (year/10*10) || '-' || (year/10*10+9) as DECADE,count(MID) as TOTALMOVIES from Movie GROUP BY [year]/10*10 order by TOTALMOVIES DESC",conn)
a.head()

,DECADE,TOTALMOVIES
0,2010-2019,1092
1,2000-2009,986
2,1990-1999,556
3,1980-1989,350
4,1970-1979,254


# answer 8

In [43]:
a=pd.read_sql_query('''SELECT   Name ,year FROM (Movie m join M_Cast c on m.MID=c.MID join PERSONS p ON p.PID=c.PID) AS e
WHERE Name not  IN (SELECT Name FROM (Movie m join M_Cast c on m.MID=c.MID join PERSONS p ON p.PID=c.PID)  as f
               WHERE  e.Name = f.Name and (e.year - f.year)*(e.year - f.year) > 9
               ORDER BY year)  order by name ''',conn)
a.head(100)

,Name,year
0,'Ganja' Karuppu,2005
1,'Ganja' Karuppu,2008
2,'Lee' George Quinones,2002
3,'Musafir' Radio Performing,2008
4,'Nandha' Saravanan,2001
5,'Om' Rakesh Chaturvedi,2018
6,'Snub' Pollard,1951
7,A'Ali de Sousa,2010
8,A. Abdul Hameed,1974
9,A. Abdul Hameed,1975


# answer 9

In [26]:
a=pd.read_sql_query('''SELECT DISTINCT(actor ) from(SELECT actor ,director,MAX(c_dir) m from
(SELECT act.Name as actor,dir.Name as director,count(*) AS c_dir 
from(M_Director d join PERSONS p ON d.PID=p.PID) as dir
JOIN ( M_Cast c JOIN PERSONS p ON c.PID=P.PID) as act ON dir.MID=act.MID GROUP BY actor,dir.Name order by c_dir desc)
as result1 group by actor order by m desc) as result2 where TRIM(director)='Yash Chopra' ''',conn)
a.head()

,actor
0,Jagdish Raj
1,Manmohan Krishna
2,Iftekhar
3,Shashi Kapoor
4,Rakhee Gulzar


# answer 10

In [27]:
a=pd.read_sql_query('''select distinct Name from PERSONS where PID in (select PID from M_Cast where MID in 
(select MID from M_Cast where PID in (select PID from M_Cast where MID in
(select MID from M_Cast c join PERSONS p ON c.PID=p.PID where Name like '%rukh Khan')))) 
and PID not in 
(select PID from M_Cast where MID in(select MID from M_Cast c join PERSONS p ON c.PID=p.PID where Name like '%rukh Khan'))

''',conn)
a.head()

,Name
0,Freida Pinto
1,Rohan Chand
2,Damian Young
3,Waris Ahluwalia
4,Caroline Christl Long
